In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from functools import reduce
from ipywidgets import *
from matplotlib.animation import FuncAnimation
import matplotlib.animation as ani
from sloppy.raytracing import *
from sloppy.abcd import *
from cavities import *

## Cavity geometry definition

## Paraxial modes and degeneracy

In [5]:
stab = lambda m: abs(0.25*np.trace(m))<0.99
La = 31.67

scanrng = 0.2
N = 300
Las = La + np.linspace(-scanrng, scanrng, N)

ms = np.zeros((N))
ws = np.zeros((N,2))
freqs = np.zeros((N,4))
for i, l in enumerate(Las):
    elements = OriginalTwister(betal=l, R=25., Rlarge=-75., thet=20., asym = 1.25)
    sys = RaySystem(elements)
    system = sys.abcd
        
    ms[i] = stab(system.abcd_rt)
    w = system.waist_at(0)
    ws[i,:] = np.sort(w)
    freqs[i,...] = np.concatenate(system.get_freqs())
    
idx = np.argmin(np.abs(freqs[:,2]))
g, ax = plt.subplots(ncols=2, figsize=(8,4))
ax[0].plot(Las, ws*1e3)
ax[0].set_ylabel('um')
ax[1].plot(Las, freqs*1e-6)
ax[1].set_ylabel('MHz')
ax[1].axhline(0, color='grey')
ax[0].axvline(Las[idx], color='grey')
ax[1].axvline(Las[idx], color='grey')
plt.show()
Ldeg = Las[idx]
print(Las[idx])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

31.674682274247495


## Paraxial eigenrays and hit pattern

In [7]:
elements = OriginalTwister(betal=Ldeg, R=25., Rlarge=-75., thet=20., asym = 1.25)
sys = RaySystem(elements)
system = sys.abcd
screen = elements[-1]

k = 2*np.pi/system.wl
mu1, mu2 = system.q

mu = mu1.real*0.05

In [8]:
def makeidx(hit_m):
    return np.arange(hit_m.shape[0])

In [9]:
hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=500)

plt.figure()
line = plt.scatter(hit_m[:,0], hit_m[:,1], c=makeidx(hit_m), cmap='jet')
plt.title('Poincare hit pattern (screen)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Raytracing

In [10]:
ray0 = screen.eigenvectors_to_rays(mu)

In [11]:
def plot_element_coordinates(el, plot, length=1.):
    #plot += k3d.vectors(el.p, length*el.xin, color=0x9633ff) #purple 
    #plot += k3d.vectors(el.p, length*el.xout, color=0xfec33) #yellow 
    plot += k3d.vectors(el.p, length*el.ax_yin, color=0xff5733) #orange
    plot += k3d.vectors(el.p, length*el.ax_yout, color=0xff33f6) #pink
    plot += k3d.vectors(el.p, length*el.n, color=0x00ff00) #green
    plot += k3d.vectors(el.p, length*el.ax_x, color=0xff0000) #red
    plot += k3d.vectors(el.p, length*el.ax_y, color=0x0000ff) #blue

In [12]:
traj0 = propagate_system(elements, ray0, Nrt=30, clip=True)

NameError: name 'propagate_system' is not defined

In [13]:
try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(elements):
    plot += el.plot(opacity=0.4)

for i, el in enumerate(elements[:-1]):
    plot_element_coordinates(el, plot, length=5.)
    
plot += k3d.vectors(screen.p, 3*screen.ax, color=0xff0000) #red
plot += k3d.vectors(screen.p, 3*screen.ay, color=0x0000ff) #blue

#plot += k3d.vectors(origins=ray0[0,...], vectors=ray0[1,...], colors=[(0xff0000,)*2], head_size=2.)
plot_trajs(traj0, plot, shader='mesh', width=0.2, color=0x00ff00)
#plot_rays(guideray, plot, length=5.)

plot.display()

/Users/mattjaffe/anaconda3/envs/sloppy/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


AttributeError: 'CurvedMirror' object has no attribute 'ax_yin'

In [15]:
traj_hit = propagate_system_at(elements, ray0, which=4, Nrt=500)[:,0,:,:] #only positions
hit_scr = screen.r_to_screen_coords(traj_hit[:,0,:])

plt.figure()
line = plt.scatter(hit_scr[:,0], hit_scr[:,1], c=makeidx(hit_scr), cmap='jet')
plt.title('Poincare hit pattern (screen)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.colorbar()
plt.show()

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\optics.py:95: RuntimeWarning: invalid value encountered in greater
  msk = np.abs(sn)>np.finfo(np.float32).eps
c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\optics.py:102: RuntimeWarning: invalid value encountered in greater
  x[(d>self.r),:] = np.nan


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
abcd_fd = extract_ABCD_fd(elements, screen, epsr=1e-3, epss=1e-3, Nrt=1) 

In [17]:
#check that the returned matrix is symplectic
G = np.array([[0,0,1,0], [0,0,0,1], [-1,0,0,0], [0,-1,0,0]])
np.round(abcd_fd.T@G@abcd_fd, decimals=5)

array([[-0.00000e+00, -2.20000e-04,  1.00008e+00,  9.00000e-05],
       [ 2.20000e-04,  0.00000e+00, -1.40000e-04,  1.00000e+00],
       [-1.00008e+00,  1.40000e-04, -0.00000e+00, -8.10000e-04],
       [-9.00000e-05, -1.00000e+00,  8.10000e-04, -0.00000e+00]])

## Main method comparison

In [18]:
fig, (ax, bx, cx) = plt.subplots(ncols=3, figsize=(11,3.3), sharex=True, sharey=True)
ms = 8.
linea = ax.scatter(hit_scr[:,0], hit_scr[:,1], c=makeidx(hit_scr), cmap='jet', s=ms)
linec = cx.scatter(hit_m[:,0], hit_m[:,1], c=makeidx(hit_m), cmap='jet', s=ms)
lineb = bx.scatter(hit_m[:,0], hit_m[:,1], c=makeidx(hit_m), cmap='jet', s=ms)
ax.set_title('Raytracing')
bx.set_title('ABCD')
cx.set_title('ABCD (Extracted)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')


def update(dl=1.267, ar=0.01, br=0.0, ap=0., bp=0., Nrt=500, **kwargs):
        elements, abcd, _ = OriginalTwister(betal=dl, R=25., Rlarge=-75., thet=20., asym = 1.25)
        screen = elements[-1]
        x0, n0 = elements[-1].p, elements[-1].n
        system = ABCDSystem(abcd)
        abcd_fd = extract_ABCD_fd(elements, screen, epsr=1e-3, epss=1e-4, Nrt=1) 

        mu1, mu2 = system.q
        mu = np.real(ar*np.exp(1j*ap)*mu1 + br*np.exp(1j*bp)*mu2)
        ray0 = screen.eigenvectors_to_rays(mu)
 
        traj_hit = propagate_system_at(elements, ray0, which=4, Nrt=Nrt)[:,0,:,:] #only get positions
        hit_scr = screen.r_to_screen_coords(traj_hit[:,0,:])

        linea.set_offsets(hit_scr[:,:])
        linea.set_array(makeidx(hit_scr))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(makeidx(hit_m))
        
        hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        linec.set_offsets(hit_fd[:,0:2])
        linec.set_array(makeidx(hit_fd))
        
        linea.set_clim(vmin=0, vmax=Nrt)
        lineb.set_clim(vmin=0, vmax=Nrt)
        linec.set_clim(vmin=0, vmax=Nrt)
            
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw_idle()
        
interactive(update, dl=(1.26, 1.30, 1e-3), ar = (0,0.1,1e-3), br = (0,0.1,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,2000,100))
#interactive(update, dl=(27.75, 27.81, 1e-3), ar = (0,0.1,1e-3), br = (0,0.1,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,2000,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.267, description='dl', max=1.3, min=1.26, step=0.001), FloatSlider(v…

In [ ]:
fig.savefig('logo.png', dpi=600)
fig.savefig('logo.svg', dpi=600)